# Exercício 1 - Esquema KEM- RSA-OAEP

Neste exercício temos de implementar um esquema KEM- RSA-OAEP  que deve :
* **Inicializar cada instância recebendo como parâmetro obrigatório o parâmetro de segurança (tamanho em bits do módulo RSA-OAEP) e gerando as chaves pública e privada**
* **Conter funções para encapsulamento e revelação da chave gerada.**
* **Construir,  a partir deste KEM e usando a transformação de Fujisaki-Okamoto, um PKE que seja IND-CCA seguro.**

Ao longo deste documento vamos explicar o algoritmo de cada um dos processos acima expostos.

## Gerar Parâmetros
Neste processo vamos gerar os parâmetros essenciais que são a base do algoritmo do RSA:
    - 2 números primos p e q;
    - n é o módulo para a chave pública e a chave privada;
    - phi é co-prime com n;

In [1]:
def rprime(l):
        return random_prime(2**l-1,True,2**(l-1))

In [2]:
l = 1024
q = rprime(l)
p = rprime(l+1)


N = p * q 
phi = (p-1)*(q-1)


## Aplicação OAEP

In [3]:
G = IntegerModRing(phi) 
R = IntegerModRing(N)


def generateKeys():
    e = G(rprime(512)) #public exponent
    s = 1/e #private exponent
    return (e,s)

e,s = generateKeys()

In [4]:
def OAEP(pk,m): ##OAEP encrypt
    a = R(m)
    cm = a**pk
    return cm
    

def OAEPinv(sk,cm): ##OAEP decrypt
    b=R(cm)
    dm = b**sk
    return dm

### Funções auxiliares para as classes KEM e FOT

In [5]:
def generateRandomString(size): # gera uma string de tamanho variável de 0 e 1 aleatórios.
    i = 0
    stream = ""
    while(i<size):
        j = randint(0,1)
        stream = stream + str(j)
        i+=1
    return stream

def generateZeroString(size): # gera uma string de tamanho variável de zeros.
    i = 0
    stream = ""
    while(i<size):
        stream = stream + str(0)
        i+=1
    return stream

xor = lambda x, y: x.__xor__(y)

def concat(i,j):
    return i +j

## KEM
Esta classe contém funções para o encapsulamento e revelação da chave gerada.

In [6]:
class KEM:
    def encrypt(self,pk,x,n):
        return power_mod(x,ZZ(pk),n)
        
    def decrypt(self,sk,enc,n):
        return power_mod(enc,ZZ(sk),n)
    
    def encapsulation(self,pk): #enc
        x = randint(1,N-1)
        #enc = self.encrypt(pk,x,N)
        enc = OAEP(pk,x)
        k = hash(x)
        return (k,enc)

    def reveal(self,sk,enc):
        #x = self.decrypt(sk,enc,N)
        x = OAEPinv(sk,enc)
        k = hash(x)
        return k
    
    def enc(self,pk):
        a = generateRandomString(l)
        zero = generateZeroString(l)
        (k,enc) = self.encapsulation1(pk,concat(a,zero))
        return (k,enc)
    
    def encapsulation1(self,pk,a0):
        enc = OAEP(pk,int(a0))
        print('encapsulation1- enc: ' + str(enc))
        k = hash(enc)
        print('encapsulation1- k: ' + str(k))
        return (k,enc)

In [7]:
kem = KEM()
(k,enc) = kem.encapsulation(e)
print('k: ' + str(k))
print('enc: ' + str(enc))

k1 = kem.reveal(s,enc)
print('k1: ' + str(k1))

teste = kem.enc(e)
print('teste: ' +str(teste))

k: 512998086484580947
enc: 27153798350488453393579947097394666019951551057479284154032211246840612741594011178026433334677928825456521577678505057884077778624181487799617226943469051577834408916714158504304016955835790092095007474764645011679425815285708504300569374443593188771949894758208634692108930324357143306879445684567016220794395280662929905453305038334005682816919605810496584608593338570933108069685194835506361959467549209987354661632816043372026338582947617633217761584932834382486477093178415605438415117580501420506887741847569785956942033826874440796755663965258983075370588470646894411424919747380178034982221641204015094381484
k1: 512998086484580947
encapsulation1- enc: 95230829828199108989102096042593688252461981668275029625359756189629616851454737001871635795224297358271168297570215419908958791577248652976257441482624160726798363251465604275803922337196800672000205507528164057417098465618692831762119624919001313103722702645161642156461964200100181781285694789740742386246137

## PKE
Esta classe implementa um esquema PKE( public key encryption) a partir do esquema KEM

In [8]:
class PKE:
    def __init__(self):
        self.kem = KEM()
    
    def encrypt(self,pk,m):
        (k,enc) = self.kem.encapsulation(pk)
        c = xor(m,k)
        return (enc,c)
    
    def decrypt(self,sk,c):
        (enc,m1) = c
        k = self.kem.reveal(sk,enc)
        m = xor(m1,k)
        return m

## FOT
Esta classe contém funções para implementar a transformação de Fujisaki-Okamoto que transforma PKE's que possuem IND-CPA seguro em outros PKE's que possuem IND-CCA seguros

In [9]:
class FOT:
    def __init__(self):
        self.kem = KEM()
        
    def encrypt(self,pk,m):
        a = generateRandomString(l)
        (enc,k) = self.encrypt1(pk,m,a)
        return (enc,k)
    
    def encrypt1(self,pk,a,m):
        (enc,k) = kem.encapsulation1(pk,concat(a,hash(m)))
        #print('encrypt1 - enc: ' + str(enc))
        #print('encrypt1 - k: ' + str(k))
        aux1 = concat(str(a),str(m))
        #print('encrypt1 - aux1: ' + str(aux1))
        aux2 = xor(int(aux1),int(k))
        #print('encrypt1 - aux2: ' + str(aux2))
        return (enc,aux2)
    
    def decrypt(self,sk,c):
        (enc,m1) = c
        k = kem.reveal(sk,enc)
        print('decrypt- k: ' + str(k))
        am = xor(m1,k)
        a = am[0:l]
        m = am[l:]
        if(c == encrypt1(pk,a,m)):
            return m
        else: 
            return false

In [ ]:
pke = PKE()

fot = FOT()
pk = e
sk = s

m = 123
c = pke.encrypt(pk,m)
(enc,t) = c
print('t: ' + str(t))
m1 = pke.decrypt(sk,c)
print('m1: ' + str(m1))

c = fot.encrypt(pk,m)
enc,k = c
print('k: '+ str(k))
c2 = fot.decrypt(sk,c)
print(c2)

### Nota

De realçar que tivemos problemas na implementação de um PKE seja um IND-CCA seguro.Mesmo assim, colocámos o código da nossa tentativa.